<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0cff4aeb9363cd1d76239710572eea4cdef7d3b2602fb6949a989f4141a58f22
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0


    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-14 09:34:27
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14


-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.40 C
-------------------
Today PnL: -19.94 K (-0.14%)
Current PnL: -28.66 L (-18.27%)
CY Booked + Current PnL: -14.37 L (-9.16%)
-------------------
Total profit:  1.78 L
Total loss:  -30.44 L
-------------------
Total Booked + Current PnL: 12.61 L (9.72%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.18%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 94.11 L (67.02%)
Deployed:  1.30 C
Current:  1.40 C
CAGR/XIRR %: 4.24%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,0.14,16.42,4.28,21.40,13225.0,43571.0,308997.0,7.98,66.0,X-LC,6.40,14.0,3.29,2.20,23.78,XY25,NTT,FMCG
17,COALINDIA,484.83,2.66,8.27,10.11,19.23,16470.0,12450.0,162911.0,27.19,73.0,L-LC,8.75,182.0,0.76,1.16,31.31,XY25,ATH,MINING
77,TTKPRESTIG,770.00,0.79,-22.10,28.49,0.09,22365.0,-22275.0,78502.0,82.28,36.0,M-SC,3.67,253.0,-1.00,0.56,3.13,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-1.32,-14.26,16.65,0.02,24920.0,-24891.0,149668.0,128.24,53.0,M-SC,8.80,240.0,-1.00,1.07,30.19,OX40N,NTT,PAINTS
49,MASFIN,398.61,0.21,-4.18,27.37,22.05,25696.0,-4095.0,93885.0,-17.34,46.0,H-SC,5.62,168.0,-0.16,0.67,36.47,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,2.66,8.27,10.11,19.23,16470.0,12450.0,162911.0,27.19,73.0,L-LC,8.75,182.0,0.76,1.16,31.31,XY25,ATH,MINING
15,CAMS,950.00,2.43,-4.58,30.45,24.48,73178.0,-11534.0,240322.0,-81.76,43.0,X-SC,3.97,86.0,-0.16,1.71,17.50,X40N,NTT,MISC
22,DIXON,18931.72,1.70,-20.76,65.65,31.26,120050.0,-47904.0,182864.0,-63.03,31.0,X-MC,20.69,56.0,-0.40,1.30,1.70,X40N,ATH,IT
40,INDUSINDBK,1800.00,1.58,-31.72,94.62,32.89,49883.0,-24488.0,52719.0,-751.34,69.0,L-MC,8.49,259.0,-0.49,0.38,45.18,XR,NTT,BANKS
36,ICICIPRULI,790.00,0.84,13.72,14.96,30.73,33514.0,27022.0,224027.0,-12.89,70.0,X-MC,7.50,79.0,0.81,1.60,28.24,X40,ATH,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-5.10,-35.56,75.21,12.90,105892.0,-77701.0,140795.0,107.93,47.0,H-SC,9.34,173.0,-0.73,1.00,17.07,OX40N,NTT,FINANCE
44,JCHAC,2282.00,-2.33,-37.18,59.17,-0.01,47506.0,-47518.0,80287.0,16334.78,40.0,M-SC,13.40,235.0,-1.00,0.57,2.76,OX40N,BTT,AC
4,ALKYLAMINE,4546.37,-2.22,-32.91,188.62,93.63,127760.0,-33229.0,67734.0,-28.23,45.0,H-SC,16.78,143.0,-0.26,0.48,2.40,SR,ATH,CHEMICALS
72,TATAELXSI,9161.00,-2.09,-18.26,61.51,32.03,66288.0,-24067.0,107768.0,-11.50,61.0,H-SC,6.62,157.0,-0.36,0.77,20.71,OX40N,NTT,IT
71,TANLA,1943.92,-1.79,-44.72,297.41,119.69,445163.0,-121081.0,149680.0,-44.15,30.0,H-SC,11.70,150.0,-0.27,1.07,18.25,AR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,0.34,-0.34,21.15,20.74,42825.0,-688.0,202482.0,31.15,55.0,M-MC,5.24,189.0,-0.02,1.44,10.24,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.17,-3.34,115.10,107.91,164258.0,-4935.0,142709.0,-21.36,48.0,M-SC,10.12,220.0,-0.03,1.02,6.41,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.32,-14.26,16.65,0.02,24920.0,-24891.0,149668.0,128.24,53.0,M-SC,8.80,240.0,-1.00,1.07,30.19,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-2.09,-18.26,61.51,32.03,66288.0,-24067.0,107768.0,-11.50,61.0,H-SC,6.62,157.0,-0.36,0.77,20.71,OX40N,NTT,IT
77,TTKPRESTIG,770.00,0.79,-22.10,28.49,0.09,22365.0,-22275.0,78502.0,82.28,36.0,M-SC,3.67,253.0,-1.00,0.56,3.13,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.03,-22.51,46.44,13.47,97042.0,-60705.0,208962.0,-68.12,47.0,H-SC,4.15,159.0,-0.63,1.49,6.25,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,2.66,8.27,10.11,19.23,16470.0,12450.0,162911.0,27.19,73.0,L-LC,8.75,182.0,0.76,1.16,31.31,XY25,ATH,MINING
84,WIPRO,420.00,-0.93,7.36,60.46,72.27,109986.0,12468.0,181916.0,-7.89,49.0,M-LC,6.30,101.0,0.11,1.30,13.99,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.93,7.36,60.46,72.27,109986.0,12468.0,181916.0,-7.89,49.0,M-LC,6.30,101.0,0.11,1.30,13.99,XR,NTT,IT
25,FINCABLES,1641.55,0.17,-3.34,115.10,107.91,164258.0,-4935.0,142709.0,-21.36,48.0,M-SC,10.12,220.0,-0.03,1.02,6.41,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.18,-8.42,125.72,106.72,142006.0,-10382.0,112954.0,-54.63,34.0,H-SC,8.03,138.0,-0.07,0.80,12.99,AR,ATH,MISC
37,IEX,219.00,-0.34,-4.42,55.51,48.63,106083.0,-8848.0,191106.0,-36.48,50.0,H-SC,13.17,137.0,-0.08,1.36,6.68,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,-0.97,-5.94,44.19,35.63,87113.0,-12442.0,197132.0,-19.40,36.0,H-MC,3.54,119.0,-0.14,1.40,9.75,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.10,-18.05,35.19,10.78,69300.0,-43386.0,196932.0,-51.37,13.0,X-LC,8.48,1.0,-0.63,1.40,0.00,X40,NTT,FMCG
7,AWL,485.00,0.15,-28.66,120.34,57.19,279978.0,-93471.0,232656.0,-66.31,22.0,X-MC,10.44,58.0,-0.33,1.66,0.15,XY24,NTT,FMCG
11,BATAINDIA,2096.00,0.17,-41.34,130.47,35.20,98484.0,-53186.0,75484.0,-1.08,28.0,X-SC,17.57,93.0,-0.54,0.54,0.17,X40,NTT,FOOTWEAR
52,PAGEIND,51605.07,-0.95,-13.15,50.39,30.62,103748.0,-31164.0,205890.0,-38.32,29.0,X-MC,16.98,57.0,-0.30,1.47,0.10,X40,ATH,APPARELS
78,UNITDSPR,1644.00,0.05,-3.03,24.54,20.77,55720.0,-7087.0,227057.0,-11.53,29.0,X-MC,1.88,70.0,-0.13,1.62,3.68,X40N,NTT,BREWERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,JIOFIN,387.00,0.32,-8.17,35.55,24.48,87286.0,-21835.0,245530.0,-9.70,37.0,X-LC,1.65,37.0,-0.25,1.75,42.32,XY24,BTT,FINANCE
78,UNITDSPR,1644.00,0.05,-3.03,24.54,20.77,55720.0,-7087.0,227057.0,-11.53,29.0,X-MC,1.88,70.0,-0.13,1.62,3.68,X40N,NTT,BREWERIES
73,TCS,4389.97,-0.93,-10.98,35.60,20.71,119865.0,-41537.0,336700.0,-23.30,51.0,X-LC,2.16,3.0,-0.35,2.40,12.50,X40,ATH,IT
20,DABUR,735.00,-0.06,2.01,40.87,43.70,103421.0,4981.0,253049.0,-6.31,64.0,X-MC,2.19,73.0,0.05,1.80,17.36,XY24,BTT,FMCG
65,SIEMENS,4671.50,0.41,-20.47,57.83,25.51,85586.0,-38100.0,147995.0,-9.21,36.0,H-LC,2.36,50.0,-0.45,1.05,9.06,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELAXO,1176.00,-0.52,-49.02,198.78,52.31,147091.0,-71163.0,73997.0,-45.70,36.0,X-SC,7.91,92.0,-0.48,0.53,0.00,X40N,NTT,FOOTWEAR
43,ITC,452.00,-0.10,-18.05,35.19,10.78,69300.0,-43386.0,196932.0,-51.37,13.0,X-LC,8.48,1.0,-0.63,1.40,0.00,X40,NTT,FMCG
52,PAGEIND,51605.07,-0.95,-13.15,50.39,30.62,103748.0,-31164.0,205890.0,-38.32,29.0,X-MC,16.98,57.0,-0.30,1.47,0.10,X40,ATH,APPARELS
53,PGHH,17907.65,0.14,-7.08,44.42,34.19,88129.0,-15120.0,198400.0,-34.66,32.0,X-MC,4.34,60.0,-0.17,1.41,0.14,X40,ATH,FMCG
7,AWL,485.00,0.15,-28.66,120.34,57.19,279978.0,-93471.0,232656.0,-66.31,22.0,X-MC,10.44,58.0,-0.33,1.66,0.15,XY24,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,0.08,-28.24,99.06,42.84,46209.0,-18359.0,46647.0,-53.17,49.0,X-SC,25.29,83.0,-0.40,0.33,5.53,XY24,NTT,MISC
58,RELAXO,1176.00,-0.52,-49.02,198.78,52.31,147091.0,-71163.0,73997.0,-45.70,36.0,X-SC,7.91,92.0,-0.48,0.53,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.17,-41.34,130.47,35.20,98484.0,-53186.0,75484.0,-1.08,28.0,X-SC,17.57,93.0,-0.54,0.54,0.17,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-0.20,-2.64,27.44,24.08,33915.0,-3354.0,123596.0,-10.46,44.0,X-SC,5.85,91.0,-0.10,0.88,16.46,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.21,-18.12,75.04,43.32,100073.0,-29512.0,133360.0,-29.66,42.0,X-SC,8.17,90.0,-0.29,0.95,2.40,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.10,-18.05,35.19,10.78,69300.0,-43386.0,196932.0,-51.37,13.0,X-LC,8.48,1.0,-0.63,1.40,0.00,X40,NTT,FMCG
75,TMPV,600.00,0.64,-27.85,70.45,22.98,114817.0,-62922.0,162976.0,-25.52,41.0,X-LC,3.72,2.0,-0.55,1.16,2.50,XY24,NTT,AUTO
73,TCS,4389.97,-0.93,-10.98,35.60,20.71,119865.0,-41537.0,336700.0,-23.30,51.0,X-LC,2.16,3.0,-0.35,2.40,12.50,X40,ATH,IT
81,VBL,671.64,0.50,1.75,33.34,35.67,107142.0,5519.0,321361.0,-10.30,61.0,X-LC,2.62,5.0,0.05,2.29,15.84,X40N,ATH,FMCG
41,INFY,1972.00,0.29,9.17,22.97,34.24,78458.0,28679.0,341567.0,-14.93,48.0,X-LC,7.59,6.0,0.37,2.43,18.21,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.58,-31.72,94.62,32.89,49883.0,-24488.0,52719.0,-751.34,69.0,L-MC,8.49,259.0,-0.49,0.38,45.18,XR,NTT,BANKS
13,BSOFT,831.70,-0.86,-23.59,97.34,50.79,104611.0,-33183.0,107470.0,-1.82,48.0,H-SC,10.19,171.0,-0.32,0.77,25.13,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.32,-14.26,16.65,0.02,24920.0,-24891.0,149668.0,128.24,53.0,M-SC,8.80,240.0,-1.00,1.07,30.19,OX40N,NTT,PAINTS
17,COALINDIA,484.83,2.66,8.27,10.11,19.23,16470.0,12450.0,162911.0,27.19,73.0,L-LC,8.75,182.0,0.76,1.16,31.31,XY25,ATH,MINING
82,VOLTAS,1856.00,-1.05,13.07,28.41,45.20,61596.0,25068.0,216810.0,2.10,56.0,X-MC,5.61,80.0,0.41,1.54,20.76,XY25,NTT,AC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,2.66,8.27,10.11,19.23,16470.0,12450.0,162911.0,27.19,73.0,L-LC,8.75,182.0,0.76,1.16,31.31,XY25,ATH,MINING
72,TATAELXSI,9161.00,-2.09,-18.26,61.51,32.03,66288.0,-24067.0,107768.0,-11.50,61.0,H-SC,6.62,157.0,-0.36,0.77,20.71,OX40N,NTT,IT
40,INDUSINDBK,1800.00,1.58,-31.72,94.62,32.89,49883.0,-24488.0,52719.0,-751.34,69.0,L-MC,8.49,259.0,-0.49,0.38,45.18,XR,NTT,BANKS
57,RECLTD,446.00,0.34,-0.34,21.15,20.74,42825.0,-688.0,202482.0,31.15,55.0,M-MC,5.24,189.0,-0.02,1.44,10.24,XY25,NTT,FINANCE
0,5PAISA,593.00,-5.10,-35.56,75.21,12.90,105892.0,-77701.0,140795.0,107.93,47.0,H-SC,9.34,173.0,-0.73,1.00,17.07,OX40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.45
1,25,44.66
2,50,76.65


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.58
MC    30.70
LC    26.76
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.34
X40      24.01
X40N     12.86
XY25     10.88
XR        8.95
AR        8.47
OX40N     7.58
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.31
X-LC    23.25
H-SC    22.73
M-SC    11.22
X-SC     7.87
H-MC     4.57
M-MC     1.44
M-LC     1.30
L-LC     1.16
H-LC     1.05
L-SC     0.76
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.31,-8.34,43.88
IT,12.85,-18.64,78.48
FINANCE,11.01,-15.36,63.78
MISC,7.07,-30.81,84.32
ELECTRICAL,5.84,-14.36,56.25
PAINTS,5.83,-12.77,29.52
INSURANCE,4.76,-1.62,36.39
PHARMA,3.95,-5.81,39.11
AUTO,2.78,-34.34,80.91


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3273015.0,21
AR,1383317.0,10
XR,1236983.0,12
X40,1149305.0,15
X40N,938171.0,9
OX40N,729119.0,10
XY25,414082.0,7
SR,287272.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3604845.0,24
X-MC,1620263.0,16
M-SC,1487134.0,15
X-LC,959284.0,13
X-SC,814961.0,8
H-MC,436608.0,3
L-SC,183419.0,2
M-LC,109986.0,1
H-LC,85586.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1260937.0      6
           AR         935905.0      5
M-SC       XY24       852343.0      6
H-SC       XR         796474.0      7
X-MC       X40        529189.0      7
           XY24       450510.0      3
X-LC       X40        425129.0      6
X-MC       X40N       417438.0      4
H-SC       OX40N      324257.0      4
X-SC       X40N       320342.0      3
M-SC       OX40N      311244.0      5
X-SC       XY24       299632.0      3
H-SC       SR         287272.0      2
H-MC       AR         229118.0      2
X-MC       XY25       223126.0      2
H-MC       XY24       207490.0      1
X-LC       XY24       202103.0      2
           X40N       200391.0      2
X-SC       X40        194987.0      2
M-SC       XR         190839.0      2
           AR         132708.0      2
X-LC       XY25       131661.0      3
M-LC       XR         109986.0      1
L-SC       OX40N       93618.0      1
           XR          89801.0      1
H-LC       AR          85586.0      1
L-MC       XR          49883.0      1
M-MC       XY25        42825.0      1
L-LC       XY25        16470.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 42.0 seconds
